In this notebook, we provide a demo for how to use the `model-equality-testing` to test if two text samples come from the same distribution.

For additional examples, please see the code used to produce the results in the paper, which can be found in the `experiments/` directory.

First, let's download the package!

In [ ]:
# !pip install model-equality-testing

## Conducting a test using two string samples

To conduct testing, we require two samples of prompt-completion pairs: one sample should be from the API we are testing, and the second sample should be from a reference distribution. 

For example, to answer if an API has modified the distribution of an open-weight model, the reference sample might be from reference model weights released on Hugging Face. To answer if the API is changing over time, the reference sample might be from the API at an earlier point in time.

In [ ]:
"""
Example samples from our dataset: these were collected for Llama-3 8B Instruct
using prompts from HumanEval. The first element of each tuple is a prompt index,
and the second is the completion as a string.
Here, we've sampled prompt-completion pairs by first sampling 20 prompts from a
uniform distribution over {0, 1}, and then sampling completions for each prompt.
"""

FP32_SAMPLE = [
    (0, 'def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    return any(abs(a - b) < threshold for a, b in zip(numbers, numbers[1:]))'),
    (1, 'Here\'s the complete code:\n\n```\nfrom typing import List\n\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    result = []\n    group = ""\n    in_group = False\n    for char in paren_string:\n        if char == "(":\n            if not in_group:\n                in_group = True\n            group += char\n        elif char == ")":\n            group += char\n            if in_group:\n                result.append(group)\n                group = ""\n                in_group = False\n    return result\n```'),
    (0, 'def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers) - 1):\n        if abs(numbers[i] - numbers[i + 1]) < threshold:\n            return True\n    return False'),
    (0, '```\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    numbers.sort()\n    for i in range(1, len(numbers)):\n        if abs(numbers[i] - numbers[i-1]) < threshold:\n            return True\n    return False'),
    (0, 'def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    numbers.sort()\n    for i in range(len(numbers) - 1):\n        if abs(numbers[i+1] - numbers[i]) < threshold:\n            return True\n    return False'),
    (0, 'def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    numbers.sort()\n    for i in range(1, len(numbers)):\n        if numbers[i] - numbers[i - 1] < threshold:\n            return True\n    return False'),
    (1, 'from typing import List\n\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    stack = []\n    groups = []\n    current_group = ""\n    \n    for char in paren_string:\n        if char == " ":\n            continue\n        if char == "(":\n            stack.append(char)\n            current_group += char\n        elif char == ")":\n            stack.pop()\n            current_group += char\n            if not stack:\n                groups.append(current_group)\n                current_group = ""\n    \n    return groups'),
    (1, "Here is the completion of the code:\n\n```\nfrom typing import List\n\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    def balance_count(s: str) -> int:\n        return sum(1 for c in s if c == '(') - sum(1 for c in s if c == ')')\n\n    groups = []\n    group = ''\n    balance = 0\n    for c in paren_string:\n        if c =='':\n            continue\n        group += c\n        balance += 1 if c == '(' else -1\n        if balance == 0:\n            groups.append(group)\n            group = ''\n            balance = 0\n\n    return groups"),
    (0, '```\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False'),
    (0, '```\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    numbers.sort()\n    for i in range(1, len(numbers)):\n        if numbers[i] - numbers[i-1] <= threshold:\n            return True\n    return False'),
    (1, "```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    stack = []\n    result = []\n    temp = ''\n    for char in(paren_string):\n        if char =='':\n            continue\n        if char == '(':\n            stack.append(char)\n            temp += char\n        elif char == ')':\n            stack.pop()\n            if not stack:\n                result.append(temp + char)\n                temp = ''\n    return result"),
    (0, '```\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```'),
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    result = []\n    stack = []\n    groups = []\n    curr_group = ""\n    for char in paren_string:\n        if char == " ":\n            continue\n        if char == "(":\n            stack.append(char)\n            curr_group += char\n        elif char == ")":\n            stack.pop()\n            curr_group += char\n            if not stack:\n                groups.append(curr_group)\n                curr_group = ""\n    return [group[1:-1] for group in groups]\n```'),
    (0, '```\n    for i in range(len(numbers) - 1):\n        if abs(numbers[i] - numbers[i+1]) < threshold:\n            return True\n    return False'),
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    result = []\n    stack = []\n    group = ""\n    \n    for char in paren_string:\n        if char == " ":\n            continue\n        if char == "(":\n            stack.append(char)\n            group += char\n        elif char == ")":\n            stack.pop()\n            group += char\n            if not stack:\n                result.append(group)\n                group = ""\n    \n    return result'),
    (1, "Here is the completed code:\n\n```\nfrom typing import List\n\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    result = []\n    temp = ''\n    for c in paren_string:\n        if c =='':\n            continue\n        temp += c\n        stack = [c]\n        i = 1\n        while i < len(temp):\n            if stack[-1] == '(' and temp[i] == ')':\n                stack.pop()\n                i += 1\n            else:\n                stack.append(temp[i])\n            i += 1\n        if not stack:\n            result.append(temp)\n            temp = ''\n    if temp:\n        result.append(temp)\n    return result"),
    (0, 'for i in range(len(numbers)):\n    for j in range(i+1, len(numbers)):\n        if abs(numbers[i] - numbers[j]) < threshold:\n            return True\nreturn False'),
    (0, 'def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    numbers.sort()\n    for i in range(1, len(numbers)):\n        if numbers[i] - numbers[i-1] <= threshold:\n            return True\n    return False'),
    (0, '```\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    numbers.sort()\n    for i in range(1, len(numbers)):\n        if numbers[i] - numbers[i - 1] < threshold:\n            return True\n    return False\n```'),
    (1, "Here is the solution:\n\n```\nfrom typing import List\n\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    groups = []\n    group = ''\n    for char in paren_string:\n        if char == '(':\n            if group:\n                groups.append(group)\n            group = ''\n        elif char == ')':\n            group += char\n        else:\n            continue\n    if group:\n        groups.append(group)\n    return groups\n```")
]

PERPLEXITY_SAMPLE = [
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    stack = []\n    groups = []\n    temp = ""\n\n    for char in paren_string:\n        if char == "(":\n            stack.append(char)\n            temp += char\n        elif char == ")":\n            if stack:\n                stack.pop()\n                temp += char\n                if not stack:\n                    groups.append(temp)\n                    temp = ""\n            else:\n                temp = ""\n        else:\n            continue\n\n    return groups'),
    (0, '```\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```'),
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    stack = []\n    groups = []\n    temp = ""\n    for char in paren_string:\n        if char == "(":\n            stack.append(char)\n            temp += char\n        elif char == ")":\n            stack.pop()\n            temp += char\n            if not stack:\n                groups.append(temp)\n                temp = ""\n    return groups'),
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    result = []\n    stack = []\n    temp = ""\n    for char in paren_string:\n        if char == " ":\n            continue\n        if char == "(":\n            stack.append(char)\n            temp += char\n        elif char == ")":\n            stack.pop()\n            temp += char\n            if not stack:\n                result.append(temp)\n                temp = ""\n    return result'),
    (0, '```\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```'),
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    stack = []\n    groups = []\n    group = ""\n    for char in paren_string:\n        if char == " ":\n            continue\n        if char == "(":\n            stack.append(char)\n            group += char\n        elif char == ")":\n            if stack:\n                stack.pop()\n                if not stack:\n                    groups.append(group + char)\n                    group = ""\n            else:\n                raise ValueError("Unbalanced parentheses")\n    if stack:\n        raise ValueError("Unbalanced parentheses")\n    return groups'),
    (0, '```\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```'),
    (0, '```\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```'),
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    stack = []\n    groups = []\n    temp = ""\n\n    for char in paren_string:\n        if char == "(":\n            stack.append(char)\n            temp += char\n        elif char == ")":\n            if stack:\n                stack.pop()\n                temp += char\n                if not stack:\n                    groups.append(temp)\n                    temp = ""\n            else:\n                temp = ""\n        else:\n            temp = ""\n\n    return groups'),
    (1, "```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    result = []\n    stack = []\n    temp = ''\n    for char in paren_string:\n        if char == '(':\n            stack.append(char)\n            temp += char\n        elif char == ')':\n            stack.pop()\n            temp += char\n            if not stack:\n                result.append(temp)\n                temp = ''\n    return result"),
    (0, '```\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```'),
    (0, '```\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```'),
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    stack = []\n    groups = []\n    temp = ""\n\n    for char in paren_string:\n        if char == "(":\n            stack.append(char)\n            temp += char\n        elif char == ")":\n            if stack:\n                stack.pop()\n                temp += char\n                if not stack:\n                    groups.append(temp)\n                    temp = ""\n            else:\n                temp = ""\n        else:\n            if stack:\n                temp += char\n            else:\n                temp = ""\n\n    return groups'),
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    stack = []\n    groups = []\n    temp = ""\n\n    for char in paren_string:\n        if char == "(":\n            stack.append(char)\n            temp += char\n        elif char == ")":\n            if stack:\n                stack.pop()\n                temp += char\n                if not stack:\n                    groups.append(temp)\n                    temp = ""\n            else:\n                temp = ""\n        else:\n            continue\n\n    return groups'),
    (0, '```\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```'),
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    stack = []\n    groups = []\n    temp = ""\n\n    for char in paren_string:\n        if char == "(":\n            stack.append(char)\n            temp += char\n        elif char == ")":\n            if stack:\n                stack.pop()\n                temp += char\n                if not stack:\n                    groups.append(temp)\n                    temp = ""\n            else:\n                return []  # invalid input\n\n    return groups'),
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    result = []\n    stack = []\n    temp = ""\n    for char in paren_string:\n        if char == " ":\n            continue\n        if char == "(":\n            stack.append(char)\n            temp += char\n        elif char == ")":\n            stack.pop()\n            temp += char\n            if not stack:\n                result.append(temp)\n                temp = ""\n    return result'),
    (0, '```\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```'),
    (0, 'for i in range(len(numbers)):\n    for j in range(i + 1, len(numbers)):\n        if abs(numbers[i] - numbers[j]) < threshold:\n            return True\nreturn False'),
    (1, '```\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    result = []\n    stack = []\n    temp = ""\n    for char in paren_string:\n        if char == " ":\n            continue\n        if char == "(":\n            stack.append(char)\n            temp += char\n        elif char == ")":\n            stack.pop()\n            temp += char\n            if not stack:\n                result.append(temp)\n                temp = ""\n    return result')
]

In [ ]:
# Tokenize these by treating each character as a unicode codepoint
# Our goal is to have for each sample: a numpy array of prompts of shape (20,) 
# and a numpy array of completions of shape (20, L)
# where L is the maximum completion length across both samples
# For padding, tokenize_unicode uses -1 as a padding token by default,
# so we use the same pad token for pad_to_length
from model_equality_testing.utils import tokenize_unicode, pad_to_length
import numpy as np

fp32_prompts, fp32_completions = zip(*FP32_SAMPLE)
fp32_completions = pad_to_length(
    tokenize_unicode(fp32_completions, pad_token_id=-1), 
    1000, 
    pad_token_id=-1
) # numpy array of shape (20, max_length)
fp32_prompts = np.array(fp32_prompts) # nump

perplexity_prompts, perplexity_completions = zip(*PERPLEXITY_SAMPLE)
perplexity_completions = pad_to_length(
    tokenize_unicode(perplexity_completions, pad_token_id=-1),
    1000,
    pad_token_id=-1
) # numpy array of shape (20, max_length)
perplexity_prompts = np.array(perplexity_prompts)

In [ ]:
# Wrap these as CompletionSample objects
# m is the total number of prompts supported by the distribution
from model_equality_testing.distribution import CompletionSample

fp32_sample = CompletionSample(prompts=fp32_prompts, completions=fp32_completions, m=2)
perplexity_sample = CompletionSample(prompts=perplexity_prompts, completions=perplexity_completions, m=2)

Once we have our samples wrapped as `CompletionSample` objects, we can run our test.

In [ ]:
from model_equality_testing.algorithm import run_two_sample_test

# Run the two-sample test
pvalue, test_statistic = run_two_sample_test(
    fp32_sample,
    perplexity_sample,
    pvalue_type="permutation_pvalue", # use the permutation procedure to compute the p-value
    stat_type="mmd_hamming", # use the MMD with Hamming kernel as the test statistic
    b=100, # number of permutations
)
print(f"p-value: {pvalue}, test statistic: {test_statistic}")
print("Reject: ", pvalue < 0.05)

This suggests that these two samples (fp32 and Perplexity) come from different underlying distributions! Compare this to what we get when running the fp32 sample against itself, where randomness comes from the permutation procedure.

In [ ]:
pvalue, test_statistic = run_two_sample_test(
    fp32_sample,
    fp32_sample,
    pvalue_type="permutation_pvalue", # use the permutation procedure to compute the p-value
    stat_type="mmd_hamming", # use the MMD with Hamming kernel as the test statistic
    b=100, # number of permutations
)
print(f"p-value: {pvalue}, test statistic: {test_statistic}")
print("Reject: ", pvalue < 0.05)

We provide other test statistics and ways of calculating pvalues: please see `model_equality_testing.tests.IMPLEMENTED_TESTS` and `model_equality_testing.pvalue.IMPLEMENTED_PVALUES` to view these.

## Loading and sampling from the dataset
To help facilitate research on tests, we release a large dataset of language model completions. To download this dataset, use the following code (requires ~38GB of space).

In [ ]:
! pip install gdown
from model_equality_testing.dataset import download_dataset
download_dataset(root_dir="./data") # will download to ./data

Once downloaded, you can load the dataset using the function `load_distribution`, which returns a `DistributionFromDataset` object. This object allows us to easily draw samples from the joint prompt-completion distribution.

In [ ]:
# load a distribution object representing the joint distribution
# where prompts come from Wikipedia (Ru) with prompt ids 0, 3, 10
# and Wikipedia (De) with prompt id 5
# and completions come from meta-llama/Meta-Llama-3-8B-Instruct
from model_equality_testing.dataset import load_distribution
p = load_distribution(
    model="meta-llama/Meta-Llama-3-8B-Instruct", # model
    prompt_ids={"wikipedia_ru": [0, 3, 10], "wikipedia_de": [5]}, # prompts
    L=1000, # number of characters to pad / truncate to
    source="fp32", # or replace with 'nf4', 'int8', 'amazon', etc.
    load_in_unicode=True, # instead of tokens
    root_dir="./data",
)

In [ ]:
sample1 = p.sample(n=100)
sample2 = p.sample(n=100)

print(sample1, sample2) # First index is the prompt index; the rest are completion tokens (unicode codepoints in this case)

In [ ]:
# Run the two-sample test
pvalue, test_statistic = run_two_sample_test(
    sample1,
    sample2,
    null_dist=p, # required for using the parametric_bootstrap pvalue_type so that we can draw additional samples from P
    pvalue_type="parametric_bootstrap", # use the simulation procedure to compute the p-value
    stat_type="mmd_hamming", # use the MMD with Hamming kernel as the test statistic
    b=100, # number of simulations
)
print(f"p-value: {pvalue}, test statistic: {test_statistic}")
print("Reject: ", pvalue < 0.05)

## Simulating power against alternatives

Finally, in the `experiments` directory we provide scripts used in the paper to simulate power against alternatives in the dataset. For example, here we run 100 simulations between the fp32 and nf4 distributions.

In [ ]:
p = load_distribution(
    model="meta-llama/Meta-Llama-3-8B-Instruct", # model
    prompt_ids={"wikipedia_ru": [0, 3, 10], "wikipedia_de": [5]}, # prompts
    L=1000, # number of characters to pad / truncate to
    source="fp32", # or replace with 'nf4', 'int8', 'amazon', etc.
    load_in_unicode=True, # instead of tokens
    root_dir="./data",
)
q = load_distribution(
    model="meta-llama/Meta-Llama-3-8B-Instruct", # model
    prompt_ids={"wikipedia_ru": [0, 3, 10], "wikipedia_de": [5]}, # prompts
    L=1000, # number of characters to pad / truncate to
    source="nf4", # or replace with 'nf4', 'int8', 'amazon', etc.
    load_in_unicode=True, # instead of tokens
    root_dir="./data",
)

In [ ]:
from experiments.testing.simulation import get_power_two_sample

power, rejections, pvalue, stat = get_power_two_sample(
    null_dist = p,
    data_dist = q,
    n_null = 100,
    n_data = 100,
    n_simulations = 10,
    alpha = 0.05,
    pvalue_type = "permutation_pvalue",
    stat_type = "mmd_hamming",
    b = 100,
    return_pvalue = True,
    return_stat = True,
)

print("Power:", power)
print("Individual reject history; true is rejected:", rejections) # one for each of the 10 simulations
print("Individual pvalue history:", pvalue)
print("Individual test statistics:", stat)

One can also cache a large number of bootstraps and reuse them later. The script `experiments/testing/cache_two_sample_bootstrap.py` does exactly this; these are then read later in `experiments/testing/simulate_two_sample_power.py`.